# Conversational Task Agent

Taskbot that assist users in multi-step tasks, such as baking a 
birthday  cake  or  fixing  a  scratch  on  a  car  —  and  adapt  those  instructions  based  on  the 
resources  and  tools  available  to  the  customer.  If,  for  example,  a  customer  ran  out  of  an 
ingredient  halfway  through  a  recipe  or  didn’t  have  a  specific  tool  for  a  DIY  project,  the 
taskbot had to adjust the plan and suggest possible solutions.

## Phase 1 - (Task Retriever): Implementing a search index for recipes with OpenSearch

## Indexing

The Indexing.py module's purpose is to handle the index operations, might these be to calculate the embeddings and interact with the pickle files used to store the embeddings, to create the index and mappings or to parse and store the recipes from the base file to OpenSearch.

In [1]:
from modules.Indexing import Indexing


indexer = Indexing()

indexer.calculateAndStoreEmbeddings()
print("Embeddings stored and encoded")
indexer.createOpenSearchMappings()
print("Mappings created")
indexer.readAndStoreRecipesFromFile("./receitas.json")
print("Recipes stored")

Embeddings stored and encoded
Index already existed. Nothing to be done.
Mappings created


SerializationError: ({'doc_id': 68, 'tags': [], 'title': 'The Only Barbecue Sauce You Need For Pork, Beef, and Anything Else You Pull off the Grill', 'title_embedding': [-0.0006707814172841609, 0.008192897774279118, 0.014633730985224247, 0.007298353593796492, 0.053565699607133865, -0.03042120300233364, -0.0979238748550415, -0.029241640120744705, 0.017199257388710976, 0.010583742521703243, 0.02843421697616577, 0.09038569778203964, -0.007841955870389938, -0.07643101364374161, 0.0312369205057621, -0.01273408718407154, -0.0004048175469506532, -0.04148063808679581, -0.06808166950941086, 0.01628856733441353, 0.011427384801208973, -0.0897151455283165, -0.047575559467077255, 0.013679955154657364, -0.02860603667795658, 0.02849041484296322, -0.02111612632870674, -0.029662707820534706, 0.0239524208009243, -0.006399562582373619, 0.027793321758508682, 0.014786746352910995, -0.010142670013010502, -0.04129337519407272, -0.06925341486930847, 0.09819823503494263, 0.07888782024383545, 0.0257881712168455

## Search

The Search.py module is simply an abstraction of the SearchBuilder.py module, made to simplify the search operations for a user. It's easily scalable thanks to the SearchBuilder layer and was made to be used as a simple interface for the system.

In [ ]:
from modules.Search import Search

search = Search()

# search.SearchTitleAndDescriptionTxt("chicken curry")
# search.SearchRecipeTime(30, 5)
# search.SearchRecipeNameTime(30, 5, "chicken")
# search.SearchRecipeIngredients("chicken curry")
# search.SearchRecipeExcludeIngredients("curry", "chicken")
# search.SearchRecipeDifficulty("easy")
# search.SearchRecipeNameDifficulty("easy", "chicken")
# search.SearchTitleEmbeddings("chicken")
# search.SearchDescriptionEmbeddings("chicken")
search.SearchSingleTitleAndDescriptionTxtInstructions("chicken curry")

## SearchBuilder

The Search Builder API is meant to be used as a tool to easily and in an iterative manner build simple or complex search OpenSearch queries. This module avoids repetitive code, also as a way to ensure good application scaling and coding practices and maintaining a developer-friendly interface.

In [3]:
import pprint as pp
from modules.SearchBuilder import SearchBuilder
from PIL import Image
import requests

searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setResultLength(5)
searchBuilder.setMandatoryTags(["Japanese"])
searchBuilder.setOptionalTags(["Sukiaki"])
searchBuilder.setIngredients("soy")
searchBuilder.excludeIngredients("chicken")
searchBuilder.setTimeFrame(150, 5)
result = searchBuilder.Search("beef Sukiyaki")
pp.pprint(result)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '876',
                    '_index': 'user212',
                    '_score': 1.8935876,
                    '_source': {'contents': "{'displayName': 'Butternut Squash "
                                            "Soup', 'description': None, "
                                            "'canonicalName': 'Butternut "
                                            "Squash Soup', 'prepTimeMinutes': "
                                            "75, 'cookTimeMinutes': None, "
                                            "'totalTimeMinutes': 75, "
                                            "'cookingMethod': None, "
                                            "'difficultyLevel': 'easy', "
                                            "'images': [{'url': "
                                            "'https://m.media-amazon.com/images/S/alexa-kitchen-msa-na-prod/recipes/foodnetwork/234e07a6582aed47fc

In [ ]:
# search for vegetarian pesto pasta
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setResultLength(5)
searchBuilder.setMandatoryTags(['Vegan', 'Vegetarian'])
searchBuilder.setOptionalTags(['Gluten Free'])

response = searchBuilder.Search("Pesto pasta")
pp.pprint(response) # print

In [ ]:
# search for Yaki udon
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setIngredients("mushrooms")
searchBuilder.excludeIngredients("brocolli")
searchBuilder.setOptionalTags(["Vegetarian"])
searchBuilder.setTimeFrame(20, 15)

response = searchBuilder.Search("Yaki udon with at home ingredients")
pp.pprint(response)# print

In [ ]:
# search for Dumpling Miso Soup Ramen
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setIngredients("dumplings")
searchBuilder.setMandatoryTags(["Vegan", "Vegetarian", "Gluten Free"])
searchBuilder.setOptionalTags(["Main Dishes", "Japanese"])

response = searchBuilder.Search("Dumpling Miso Soup Ramen")
pp.pprint(response)# print

In [ ]:
# search for Ceasar Salad
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setOptionalTags(["Main Dishes"])

response = searchBuilder.SearchByDescriptionEmbeddings("Ceasar Salad")
pp.pprint(response)# print

In [2]:
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()

lasagna_url = "https://www.unileverfoodsolutions.pt/dam/global-ufs/mcos/portugal/calcmenu/recipes/PT-recipes/In-Development/lasanha/main-header.jpg"

response = searchBuilder.SearchByImageEmbeddings([Image.open(requests.get(lasagna_url, stream=True).raw)])
pp.print(response)

[[-2.07018666e-02  2.85040438e-02 -1.45819811e-02 -1.09657124e-02
   5.65259892e-04  4.04812694e-02 -6.02038093e-02  7.15839714e-02
   3.27181257e-02 -2.85275429e-02 -3.16352956e-02 -5.20524383e-02
   5.46285063e-02  4.11662972e-03  6.57309592e-02  1.08181089e-02
   6.24553487e-02 -1.34481175e-03  3.12445331e-02 -2.63105500e-02
  -1.14707127e-01  2.80214772e-02  7.86186720e-05  6.89829793e-03
   1.49037223e-02  3.81790251e-02  2.93273013e-02  2.29963940e-02
  -1.70767941e-02  9.34916642e-03 -5.14225161e-04 -1.25861987e-02
   2.91597936e-02 -4.38077040e-02  4.47314717e-02 -1.22992964e-02
   2.26408523e-02 -1.60304680e-02 -6.95216954e-02  1.82884648e-01
  -2.27763243e-02 -9.99534037e-03  1.23636322e-02 -1.03110587e-02
  -4.55162711e-02 -8.92882273e-02  2.11238693e-02  5.06581506e-03
   6.43370487e-03 -7.09157251e-03 -1.80167016e-02  8.49639066e-03
  -9.77515616e-03  2.37212889e-02  1.58446059e-02  2.67330557e-02
   1.76805798e-02 -4.05567251e-02  3.85358813e-03 -5.63285910e-02
  -3.68470

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [3]:
from opensearchpy import OpenSearch

host = 'api.novasearch.org'
port = 443
user = 'user212'
password = 'soO-2518'
index_name = user
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

## Closing the client connection

In [ ]:
be sure you want to close the connection before running this code

resp = client.indices.close(index = index_name, timeout=600)
print(resp)

## Deleting the Index

In [4]:
be sure you want to delete the index before running this code

if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = 600
    )
    print('\nDeleting index:')
    print(response)


Deleting index:
{'acknowledged': True}
